# Setup

## Dependencies

In [20]:
# import data_prep
import numpy as np
import pandas as pd

## Importing Data

In [21]:
############### PROJECT TABLE DATA ####################
project_table_data = "./data/database_tables/project_table.csv"
project_table = pd.read_csv(
    project_table_data, 
    dtype={'Claim #': str,'Job #': str,'Branch':str,'Claim Status':str},
    parse_dates=[
        'Rep Agreement Signed', 'Rep Claim Collected','FTA Scope Completed',
        'FTA Scope Rejected', 'BC Estimate Completed','OB Scope Completed',
        'Sup Job Submitted', 'BC Approved for Production', 'OB Order Built',
        'GM Order Processed', 'PA Permit Applied', 'PA Permit Processed',
        'PA OA Processed', 'PA OA Invoiced', 'PA Notify of Delivery',
        'PA Notify of Start', 'Delivery Date', 'Roof Start',
        'Roof End', 'GM Approved for Inspection', 'GM Change Order Date',
        'GM Labor Adjustment Date', 'RA Inspection Requested', 'RA Inspection Processed', 
        'Rep COC Collected', 'SA Job Docs Uploaded', 'BC Project Invoiced','BC Project Closed'])

############### PROJECT INFORMATION TABLE DATA ###############
project_info_data = "./data/database_tables/project_info_table.csv"

info_table = pd.read_csv(project_info_data, dtype={
    'Claim #':str, 'Job #':str, 'Branch':str, 'City':str, 'Building Department':str,
    'Permit Req?':str, 'Supplier Name':str, 'Crew':str, 'Insurance Company':str,
    'Multi-rejected':str, 'Scope Rejections':str, 'Change Orders':str,'Labor Adjustments':str})

# due to manual 'city' field, city is incorrectly spelled or off on capitalization
info_table['City'] = info_table['City'].str.upper()

#################### WORKFLOW TABLE DATA ####################
workflow_table_data = "./data/database_tables/workflow_table.csv"
workflow_table = pd.read_csv(workflow_table_data)

######################### EAGLEVIEW DATA #########################
eagleview_table_data = "./data/raw_datasets/[TVA] EagleView Analysis.csv"
eagleview_table = pd.read_csv(eagleview_table_data)

# converting the squares measurement to roofing SQs
eagleview_table['Square Feet'] = eagleview_table['Square Feet'] / 100





info_table_data = info_table[['Job #', 'Permit Req?']]
filtered_info_table_data = info_table_data.loc[(info_table_data['Job #'].isnull() == False ),:]
pipeline_data = project_table.merge(filtered_info_table_data, how='left', on='Job #')

# IP Code

In [23]:
pipeline_dict = {  
    'Rep Claims': {'start':'Rep Agreement Signed', 'end':'Rep Claim Collected'}, 
    'FTA Scopes': {'start':'Rep Claim Collected', 'end':'FTA Scope Completed'}, 
    'BC Estimates': {'start':'FTA Scope Completed', 'end':'BC Estimate Completed'},
    'OB Scopes': {'start':'BC Estimate Completed', 'end':'OB Scope Completed'},
    'Sup Submittals': {'start':'OB Scope Completed', 'end':'Sup Job Submitted'},
    'BC Approvals': {'start':'Sup Job Submitted', 'end':'BC Approved for Production'},
    'OB Orders Built': {'start':'BC Approved for Production', 'end':'OB Order Built'},
    'GM Orders Processed': {'start':'OB Order Built', 'end':'GM Order Processed'},
    'PA Permits Applied': {'start':'GM Order Processed', 'end':'PA Permit Applied'},
    'PA Permits Processed':{'start':'PA Permit Applied', 'end':'PA Permit Processed'},
    'PA OA Processed':{'start':'GM Order Processed', 'end':'PA OA Processed'},
    'PA OA Invoiced':{'start':'PA OA Processed', 'end':'PA OA Invoiced'},
    'PA Notify of Delivery':{'start':'Delivery Date', 'end':'PA Notify of Delivery'},
    'PA Notify of Start':{'start':'Roof Start', 'end':'PA Notify of Start'}, 
    'GM Approved for Inspection':{'start':'Roof End', 'end':'GM Approved for Inspection'},
    'RA Inspection Requested':{'start':'GM Approved for Inspection', 'end':'RA Inspection Requested'},
    'RA Inspection Processed':{'start':'RA Inspection Requested', 'end':'RA Inspection Processed'},
    'Rep COC Collected':{'start':'Roof End', 'end':'Rep COC Collected'},
    'SA Job Docs Uploaded':{'start':'Rep COC Collected', 'end':'SA Job Docs Uploaded'},
    'BC Project Invoiced':{'start':'SA Job Docs Uploaded', 'end':'BC Project Invoiced'},
    'BC Project Closed':{'start':'BC Project Invoiced', 'end':'BC Project Closed'},
}


class Branch_Pipeline:

    def __init__(self, branch, pipeline):
        self.branch = branch
        self.pipeline = pipeline

    ###
    def selected_branch(self):
        
        pipeline_date = pipeline_dict.get(self.pipeline)
        start = pipeline_date['start']
        end = pipeline_date['end']
        # need to add the 'additional' or third field here

        if self.branch == 'All':
            pipeline_data = project_table

        else:
            pipeline_data = project_table.loc[(project_table['Branch'] == self.branch), :]

            
        branch_data = pipeline_data.loc[(
            pipeline_data[start].isnull() == False )&(
            pipeline_data[end].isnull() == True),:]
        
        print(f'Branch being Analyzed: {self.branch}')
        print(f'Pipeline Chosen: {self.pipeline}')
        
        
        return branch_data.describe()
    

def run_analysis():
    
    print('''
    #########################################################################################

                                        POSSIBLE BRANCHES

                                ALL | DEN | COS | FCO | KCI | OMA

    #########################################################################################
    ''')
    
    branch_response = input('Choose your Branch: ')
    
    
    print('''
    
    #################################################################
    
                            POSSIBLE PIPELINES

    Rep Claims | FTA Scopes| BC Estimates | OB Scopes | Sup Submittals
    
    BC Approvals | OB Orders Built | GM Orders Processed | PA Permits Applied
    
    PA Permits Processed | PA OA Processed | PA OA Invoiced | PA Notify of Delivery
    
    PA Notify of Start | GM Approved for Inspection | RA Inspection Requested
    
    RA Inspection Processed | Rep COC Collected | SA Job Docs Uploaded 
    
    BC Project Invoiced | BC Project Closed
    
    ######################################################################
    
    ''')
    
    pipeline_response = input("Generate which Pipeline?: ")

    analysis = Branch_Pipeline(branch_response, pipeline_response)
    
    
    return analysis.selected_branch()

run_analysis()


    #########################################################################################

                                        POSSIBLE BRANCHES

                                ALL | DEN | COS | FCO | KCI | OMA

    #########################################################################################
    
Choose your Branch: DEN

    
    #################################################################
    
                            POSSIBLE PIPELINES

    Rep Claims | FTA Scopes| BC Estimates | OB Scopes | Sup Submittals
    
    BC Approvals | OB Orders Built | GM Orders Processed | PA Permits Applied
    
    PA Permits Processed | PA OA Processed | PA OA Invoiced | PA Notify of Delivery
    
    PA Notify of Start | GM Approved for Inspection | RA Inspection Requested
    
    RA Inspection Processed | Rep COC Collected | SA Job Docs Uploaded 
    
    BC Project Invoiced | BC Project Closed
    
    #################################################################

,Claim #,Job #,Branch,Claim Status,Rep Agreement Signed,Rep Claim Collected,FTA Scope Completed,FTA Scope Rejected,BC Estimate Completed,OB Scope Completed,...,Roof End,GM Approved for Inspection,GM Change Order Date,GM Labor Adjustment Date,RA Inspection Requested,RA Inspection Processed,Rep COC Collected,SA Job Docs Uploaded,BC Project Invoiced,BC Project Closed
count,44,44,44,0.0,44,44,44,31,44,44,...,44,44,18,14,38,33,44,44,44,0.0
unique,44,44,1,0.0,31,29,30,23,30,31,...,31,24,9,9,23,17,28,18,19,0.0
top,068335V81,1936242,DEN,NaN,2019-06-13 00:00:00,2019-06-20 00:00:00,2019-05-07 00:00:00,2019-05-08 00:00:00,2019-06-12 00:00:00,2019-07-12 00:00:00,...,2019-07-31 00:00:00,2019-08-06 00:00:00,2019-08-06 00:00:00,2019-07-30 00:00:00,2019-07-31 00:00:00,2019-08-01 00:00:00,2019-08-05 00:00:00,2019-07-30 00:00:00,2019-08-14 00:00:00,NaN
freq,1,1,44,NaN,3,4,4,3,3,4,...,3,7,4,4,4,4,5,8,7,NaN
first,NaN,NaN,NaN,NaN,2019-03-26 00:00:00,2019-03-29 00:00:00,2019-04-05 00:00:00,2019-04-09 00:00:00,2019-04-09 00:00:00,2019-04-09 00:00:00,...,2019-05-03 00:00:00,2019-05-03 00:00:00,2019-06-04 00:00:00,2019-06-11 00:00:00,2019-05-22 00:00:00,2019-05-24 00:00:00,2019-05-15 00:00:00,2019-06-13 00:00:00,2019-06-19 00:00:00,NaN
last,NaN,NaN,NaN,NaN,2019-07-15 00:00:00,2019-07-15 00:00:00,2019-07-17 00:00:00,2019-07-15 00:00:00,2019-07-19 00:00:00,2019-07-22 00:00:00,...,2019-08-09 00:00:00,2019-08-13 00:00:00,2019-08-13 00:00:00,2019-08-13 00:00:00,2019-08-12 00:00:00,2019-08-14 00:00:00,2019-08-13 00:00:00,2019-08-13 00:00:00,2019-08-14 00:00:00,NaN


In [22]:
# these are the pipelines that require the 3rd additional filter
ob_scope_pipeline = pipeline_data.loc[(
    pipeline_data['BC Estimate Completed'].isnull() == False )&(
    pipeline_data['OB Scope Completed'].isnull() == True)&(
    pipeline_data['FTA Scope Rejected'].isnull() == True),:]

sup_submit_pipeline = pipeline_data.loc[(
    pipeline_data['OB Scope Completed'].isnull() == False )&(
    pipeline_data['Sup Job Submitted'].isnull() == True)&(
    pipeline_data['Job #'].isnull() == True),:]

pa_permit_applied_pipeline = pipeline_data.loc[(
    pipeline_data['GM Order Processed'].isnull() == False )&(
    pipeline_data['PA Permit Applied'].isnull() == True)&(
    pipeline_data['Permit Req?'] == 'Y'),:]

pa_permit_processed_pipeline = pipeline_data.loc[(
    pipeline_data['PA Permit Applied'].isnull() == False )&(
    pipeline_data['PA Permit Processed'].isnull() == True)&(
    pipeline_data['Permit Req?'] == 'Y'),:]

gm_approved_inspection_pipeline = pipeline_data.loc[(
    pipeline_data['Roof End'].isnull() == False )&(
    pipeline_data['GM Approved for Inspection'].isnull() == True)&(
    pipeline_data['Permit Req?'] == 'Y'),:]

ra_inspection_requested_pipeline = pipeline_data.loc[(
    pipeline_data['GM Approved for Inspection'].isnull() == False )&(
    pipeline_data['RA Inspection Requested'].isnull() == True)&(
    pipeline_data['Permit Req?'] == 'Y'),:]

ra_inspection_processed_pipeline = pipeline_data.loc[(
    pipeline_data['RA Inspection Requested'].isnull() == False )&(
    pipeline_data['RA Inspection Processed'].isnull() == True)&(
    pipeline_data['Permit Req?'] == 'Y'),:]